In [40]:
library(data.table)
library(tidyverse)
library(pheatmap)

In [486]:
# testing data
test_meta_data <- fread("../Maize_GxE_Competition_Data/Testing_Data/2_Testing_Meta_Data_2022.csv")
test_soil_data <- fread("../Maize_GxE_Competition_Data/Testing_Data/3_Testing_Soil_Data_2022.csv")
test_ec_data <- fread("../Maize_GxE_Competition_Data/Testing_Data/6_Testing_EC_Data_2022.csv")
submission_template <- fread("../Maize_GxE_Competition_Data/Testing_Data/1_Submission_Template_2022.csv")
weather_data_filtered_test <- fread("4_Testing_Weather_Data_2022_tsfresh_extracted_features_filtered.csv")

In [487]:
# training data
trait_data <- fread("../Maize_GxE_Competition_Data/Training_Data/1_Training_Trait_Data_2014_2021.csv")
meta_data <- fread("../Maize_GxE_Competition_Data/Training_Data/2_Training_Meta_Data_2014_2021.csv")
soil_data <- fread("../Maize_GxE_Competition_Data/Training_Data/3_Training_Soil_Data_2015_2021.csv")
ec_data <- fread("../Maize_GxE_Competition_Data/Training_Data/6_Training_EC_Data_2014_2021.csv")
weather_data_filtered <- fread("4_Training_Weather_Data_2014_2021_tsfresh_extracted_features_filtered_2.csv")
ec_relevant <- fread("EC_feature_relevance.csv")
prs_10 <- fread("maize_trait_selected_10_prs_scores.txt")
prs_50 <- fread("maize_trait_selected_50_prs_scores.txt")
prs_100 <- fread("maize_trait_selected_100_prs_scores.txt")
prs_200 <- fread("maize_trait_selected_200_prs_scores.txt")

## Meta Data EDA

### training data wrangling

In [488]:
## change of meta_data
meta_treatment <- as.factor(meta_data$Treatment)
levels(meta_treatment) <- c('','Disease trial','Drought', 'Dryland','Dryland',
                           'Dryland optimal', 'Early Planting','Irrigated',
                           'Late Planting Irrigated', 'Late Planting',
                           'Late Stressed', 'Standard',
                           'Standard - Irrigated Optimal')
meta_data$Treatment<- as.character(meta_treatment)

meta_city <- as.factor(meta_data$City)
levels(meta_city) <-  c('','Ames','Arlington','Aurora','Brule','Carroll','Champaign',
                        'Clayton','Clemson','College Station', 'College Station',
                        'College Station','Columbia','Crawfordsville','East Lansing',
                        'Fort Collins', 'Garden City','Georgetown','Gottingen','Halfway',
                        'Hancock','Jonesboro','Keiser','Keystone','Kinston','Lewiston',
                        'Lincoln','Lincoln','Lubbock','Madison','Manhattan','Marianna','Mead',
                        'New Underwood','Newark','North Platte','Pendleton','Ridgetown','South Charleston',
                        'Tifton','Urbana','Wahoo','Waseca','Waterloo','Watkinsville','West Lafayette')
meta_data$City <- as.character(meta_city)

meta_prev_crop <- as.factor(meta_data$Previous_Crop)
levels(meta_prev_crop) <- c('','wheat','corn','cotton','Fallow/Glystar','Lima beans/rye cover crop',
                            'peanut','wheat/soybean','sorghum','soybean',
                            'soybean/pumpkin', 'sugar beet','wheat','wheat/soybean',
                            'wheat/soybean','wheat/soybean','wheat')
meta_data$Previous_Crop <- as.character(meta_prev_crop)

meta_moisture_determ <- as.factor(meta_data_pared$System_Determining_Moisture)
unique_levels <- tolower(levels(meta_moisture_determ)[-1])

levels(meta_moisture_determ) <- c('', gsub('almaco.*','almaco',
     gsub('harvest.*|.* grain gauge.*|^grain .*|juniper|mirus.*|myrris', 'grain gauge',unique_levels)))
meta_data_pared$System_Determining_Moisture <- as.character(meta_moisture_determ)

In [489]:
percent_missing <- function(x) {
    return(sum(is.na(x) | x == "")/length(x))
}

meta_percent_missing <- apply(meta_data,2,percent_missing)
meta_percent_missing_ordered <- meta_percent_missing[order(meta_percent_missing, decreasing = T)]
meta_percent_missing_ordered
removal_high_percent_missing <- names(meta_percent_missing_ordered[meta_percent_missing_ordered > .35])

Issue/comment_#6 
                                                        0.99539171 
                                                  Issue/comment_#5 
                                                        0.96774194 
                                                          Comments 
                                                        0.95391705 
                                                  Issue/comment_#4 
                                                        0.93087558 
                                                  Issue/comment_#3 
                                                        0.90322581 
                                                  Issue/comment_#2 
                                                        0.81105991 
               Soil_Taxonomic_ID and horizon description, if known 
                                                        0.72350230 
                                       In-season_tillage_method(s) 
                                                        0.70506912 
                                                  Issue/comment_#1 
                                                        0.66359447 
        Trial_ID (Assigned by collaborator for internal reference) 
                                                        0.52534562 
                                           Cardinal_Heading_Pass_1 
                                                        0.45161290 
                                                             Field 
                                                        0.40092166 
                                       Pounds_Needed_Soil_Moisture 
                                                        0.36866359 
                                      Date_weather_station_removed 
                                                        0.35483871 
                                       Date_weather_station_placed 
                                                        0.32258065 
                                       System_Determining_Moisture 
                                                        0.30875576 
                                                         Treatment 
                                                        0.25806452 
                                                              Farm 
                                                        0.25806452 
                         Latitude_of_Field_Corner_#2 (lower right) 
                                                        0.24884793 
                        Longitude_of_Field_Corner_#2 (lower right) 
                                                        0.24884793 
                         Latitude_of_Field_Corner_#3 (upper right) 
                                                        0.24884793 
                        Longitude_of_Field_Corner_#3 (upper right) 
                                                        0.24884793 
                          Latitude_of_Field_Corner_#4 (upper left) 
                                                        0.24423963 
                         Longitude_of_Field_Corner_#4 (upper left) 
                                                        0.24423963 
                          Latitude_of_Field_Corner_#1 (lower left) 
                                                        0.23963134 
                         Longitude_of_Field_Corner_#1 (lower left) 
                                                        0.23963134 
                                       Pre-plant_tillage_method(s) 
                                                        0.20737327 
                                                     Previous_Crop 
                                                        0.14285714 
             Type_of_planter (fluted cone; belt cone; air planter) 
                                                        0.11520737 
             Weather_Station_Latitude (in decimal numbers NOT DMS) 
                                      

In [490]:
# remove unimportant columns and columns with greater than 35% missing data
final_meta_data <- meta_data[,-c('Experiment_Code',
                                 'Date_weather_station_removed',
                                 'Date_weather_station_placed',
                                 'Farm',
                                 "Year",
                                'Cardinal_Heading_Pass_1',
                                 'Issue/comment_#1','Issue/comment_#2',
                                 'Issue/comment_#3',
                                 'Issue/comment_#4',
                                 'Issue/comment_#5',
                                 'Issue/comment_#6',
                                 'Comments',
                                 'System_Determining_Moisture',
                                 'Soil_Taxonomic_ID and horizon description, if known'
                                 )]
head(final_meta_data)

Env,Treatment,City,Field,Trial_ID (Assigned by collaborator for internal reference),"Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)",Weather_Station_Latitude (in decimal numbers NOT DMS),Weather_Station_Longitude (in decimal numbers NOT DMS),Previous_Crop,Pre-plant_tillage_method(s),⋯,Type_of_planter (fluted cone; belt cone; air planter),Pounds_Needed_Soil_Moisture,Latitude_of_Field_Corner_#1 (lower left),Longitude_of_Field_Corner_#1 (lower left),Latitude_of_Field_Corner_#2 (lower right),Longitude_of_Field_Corner_#2 (lower right),Latitude_of_Field_Corner_#3 (upper right),Longitude_of_Field_Corner_#3 (upper right),Latitude_of_Field_Corner_#4 (upper left),Longitude_of_Field_Corner_#4 (upper left)
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TXH1-Early_2017,Standard,College Station,210,CS17-G2FE,,30.54535,-96.43258,,,⋯,,,NA,NA,NA,NA,NA,NA,NA,NA
TXH1-Late_2017,Standard,College Station,210,CS17-G2FE,,30.54535,-96.43258,,,⋯,,,NA,NA,NA,NA,NA,NA,NA,NA
IAH1b_2014,,Ames,,,9080,41.99653,-93.69619,soybean,field cultivator,⋯,Air planter,,NA,NA,NA,NA,NA,NA,NA,NA
IAH1c_2014,,Ames,,,9080,41.99653,-93.69619,soybean,field cultivator,⋯,Air planter,,NA,NA,NA,NA,NA,NA,NA,NA
ARH1_2016,,Marianna,,Mariana,8658,34.72833,-90.76028,corn,"Cultivate, hip and row",⋯,fluted cone,,34.72988,-90.75974,34.72989,-90.76094,34.72915,-90.75982,34.72915,-90.76092
ARH1_2017,Standard,Marianna,,,8658,34.72837,-90.75980,corn,disk and hip,⋯,John Deere 7300,4,34.72987,-90.75977,34.72986,-90.76089,34.72014,-90.76088,34.72913,-90.75977


In [491]:
dim(final_meta_data)

[1] 217  21

### testing data wrangling

In [492]:
# remove unimportant columns and columns with greater than 35% missing data
final_test_meta_data <- select(test_meta_data, colnames(test_meta_data)[colnames(test_meta_data) %in% colnames(final_meta_data)])

## Soil Data EDA

### training data wrangling

In [493]:
# select only ward laboratories data
ward_soil_data <- soil_data[soil_data$LabID == "Ward Laboratories Inc", 1:28]
soil_data_final_columns <- ward_soil_data[,-c('LabID', "Year",'Date Received', 'Date Reported')]
dim(soil_data_final_columns)

## normalize the soil data by the maximum value
final_soil_data <- data.frame(apply(soil_data_final_columns[,2:length(soil_data_final_columns)],
                         2, FUN=function(x){(x/max(x, na.rm = T))})
)
head(final_soil_data)
dim(final_soil_data)

[1] 124  24

,E.Depth,X1.1.Soil.pH,WDRF.Buffer.pH,X1.1.S.Salts.mmho.cm,Texture.No,Organic.Matter.LOI..,Nitrate.N.ppm.N,lbs.N.A,Potassium.ppm.K,Sulfate.S.ppm.S,⋯,CEC.Sum.of.Cations.me.100g,X.H.Sat,X.K.Sat,X.Ca.Sat,X.Mg.Sat,X.Na.Sat,Mehlich.P.III.ppm.P,X..Sand,X..Silt,X..Clay
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1.0000000,0.6904762,0.9444444,0.3421053,1.0,0.4225352,0.4064698,0.5252525,0.2094017,0.04499761,⋯,0.2660333,0.2463768,0.2142857,0.6666667,0.4722222,0.5,0.2675159,0.2906977,0.79104478,0.3333333
2,1.0000000,0.7261905,0.9583333,0.4736842,1.0,0.3380282,0.7721519,1.0000000,0.1424501,0.04547630,⋯,0.2779097,0.1014493,0.1428571,0.8064516,0.4166667,0.5,0.1337580,0.2209302,1.00000000,0.2121212
3,1.0000000,0.6785714,0.8888889,0.4868421,1.0,0.3239437,0.4838256,0.6262626,0.2478632,0.18190522,⋯,0.2232779,0.8550725,0.3571429,0.2795699,0.2777778,0.0,1.0000000,0.8720930,0.19402985,0.1818182
4,1.0000000,0.7142857,0.9166667,0.2894737,1.0,0.2253521,0.2686357,0.3484848,0.1709402,0.15222595,⋯,0.1543943,0.8985507,0.3571429,0.2688172,0.1944444,0.5,0.5541401,0.9069767,0.14925373,0.1818182
5,0.6666667,0.7857143,1.0000000,0.1710526,0.5,0.3380282,0.4022504,0.3484848,0.1780627,0.14360938,⋯,0.0807601,0.0000000,0.6428571,0.7634409,0.5000000,0.5,0.8662420,0.9418605,0.13432836,0.1515152
6,0.6666667,0.7261905,0.9444444,0.2368421,1.0,0.2112676,0.4683544,0.4040404,0.1666667,0.09191000,⋯,0.1258907,0.6231884,0.4285714,0.4516129,0.2222222,0.5,0.6878981,0.9767442,0.08955224,0.1515152


[1] 124  23

In [494]:
# write out to imputer
fwrite(final_soil_data, "normalized_soil.csv")

In [495]:
# save imputed soil data
imputed_soil_data<- fread("iputed_soil_data.csv")
final_soil_data <- data.frame("Env" = soil_data_final_columns$Env,imputed_soil_data[,2:length(imputed_soil_data)])
colnames(final_soil_data) <- colnames(soil_data_final_columns)

In [496]:
final_soil_data

Env,E Depth,1:1 Soil pH,WDRF Buffer pH,1:1 S Salts mmho/cm,Texture No,Organic Matter LOI %,Nitrate-N ppm N,lbs N/A,Potassium ppm K,⋯,CEC/Sum of Cations me/100g,%H Sat,%K Sat,%Ca Sat,%Mg Sat,%Na Sat,Mehlich P-III ppm P,% Sand,% Silt,% Clay
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NEH3_2021,1.0000000,0.6904762,0.9444444,0.34210526,1.0,0.4225352,0.40646976,0.52525253,0.2094017,⋯,0.26603325,0.2463768,0.21428571,0.6666667,0.4722222,0.5,0.26751592,0.2906977,0.79104478,0.33333333
MOH1_1_2020,1.0000000,0.7261905,0.9583333,0.47368421,1.0,0.3380282,0.77215190,1.00000000,0.1424501,⋯,0.27790974,0.1014493,0.14285714,0.8064516,0.4166667,0.5,0.13375796,0.2209302,1.00000000,0.21212121
DEH1_2016,1.0000000,0.6785714,0.8888889,0.48684211,1.0,0.3239437,0.48382560,0.62626263,0.2478632,⋯,0.22327791,0.8550725,0.35714286,0.2795699,0.2777778,0.0,1.00000000,0.8720930,0.19402985,0.18181818
DEH1_2017,1.0000000,0.7142857,0.9166667,0.28947368,1.0,0.2253521,0.26863572,0.34848485,0.1709402,⋯,0.15439430,0.8985507,0.35714286,0.2688172,0.1944444,0.5,0.55414013,0.9069767,0.14925373,0.18181818
DEH1_2019,0.6666667,0.7857143,1.0000000,0.17105263,0.5,0.3380282,0.40225035,0.34848485,0.1780627,⋯,0.08076010,0.0000000,0.64285714,0.7634409,0.5000000,0.5,0.86624204,0.9418605,0.13432836,0.15151515
DEH1_2020,0.6666667,0.7261905,0.9444444,0.23684211,1.0,0.2112676,0.46835443,0.40404040,0.1666667,⋯,0.12589074,0.6231884,0.42857143,0.4516129,0.2222222,0.5,0.68789809,0.9767442,0.08955224,0.15151515
DEH1_2021,0.6666667,0.7261905,0.9444444,0.34210526,1.0,0.1408451,0.62025316,0.53535354,0.1638177,⋯,0.09738717,0.5217391,0.50000000,0.4731183,0.3333333,0.5,0.65605096,0.3604651,0.65671642,0.37878788
GAH1_2016,1.0000000,0.7857143,1.0000000,0.42105263,1.0,0.1408451,0.40928270,0.53030303,0.2336182,⋯,0.08551069,0.0000000,0.85714286,0.8279570,0.2777778,0.5,0.81528662,0.9418605,0.16417910,0.12121212
GAH1_2017,1.0000000,0.7857143,1.0000000,0.07894737,1.0,0.1408451,0.01969058,0.02525253,0.1353276,⋯,0.08076010,0.0000000,0.50000000,0.8172043,0.4444444,0.5,0.78343949,0.9186047,0.16417910,0.15151515


### testing data wrangling

In [497]:
test_soil_unnormalized <- select(test_soil_data, colnames(test_soil_data)[colnames(test_soil_data) %in% colnames(final_soil_data)])

## normalize the soil data by the maximum value
final_test_soil_data_normalized <- data.frame(apply(test_soil_unnormalized[,2:length(test_soil_unnormalized)],
                         2, FUN=function(x){(x/max(x, na.rm = T))})
)
head(final_test_soil_data)
dim(final_test_soil_data)

,Env,E Depth,1:1 Soil pH,WDRF Buffer pH,1:1 S Salts mmho/cm,Texture No,Organic Matter LOI %,Nitrate-N ppm N,lbs N/A,Potassium ppm K,⋯,CEC/Sum of Cations me/100g,%H Sat,%K Sat,%Ca Sat,%Mg Sat,%Na Sat,Mehlich P-III ppm P,% Sand,% Silt,% Clay
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,DEH1_2022,0.6666667,0.7951807,1.0000000,0.4038462,1,0.2448980,0.5363036,0.4406780,0.4579125,⋯,0.1067073,0.0000000,0.7692308,0.7582418,0.5757576,1.0,0.5474453,0.8915663,0.2727273,0.2820513
2,GAH1_2022,1.0000000,0.7469880,0.9583333,1.0000000,1,0.1836735,0.4884488,0.6045198,0.7946128,⋯,0.1463415,0.5087719,1.0000000,0.4835165,0.3939394,0.5,1.0000000,1.0000000,0.1818182,0.1794872
3,GAH2_2022,1.0000000,0.7349398,0.9305556,0.4423077,1,0.5306122,0.3118812,0.3841808,0.5016835,⋯,0.2225610,0.7894737,0.3846154,0.4175824,0.3333333,0.5,0.3284672,0.7590361,0.2909091,0.5384615
4,IAH1_2022,0.6666667,0.7469880,0.9166667,0.6923077,1,1.0000000,0.6056106,0.4971751,0.6632997,⋯,0.7530488,0.2631579,0.1538462,0.6483516,0.7272727,0.0,0.3722628,0.1927711,0.8181818,1.0000000
5,IAH2_2022,0.6666667,0.7108434,0.8888889,0.3269231,1,0.7959184,0.2062706,0.1694915,0.6599327,⋯,0.5762195,0.5263158,0.2307692,0.6043956,0.3636364,0.0,0.4306569,0.5301205,0.5272727,0.6923077
6,IAH3_2022,0.6666667,0.7951807,1.0000000,0.5192308,1,0.9795918,0.4339934,0.3559322,0.6531987,⋯,0.5762195,0.0000000,0.2307692,0.8571429,0.5757576,0.0,0.3430657,0.2168675,0.8545455,0.8974359


[1] 21 24

In [498]:
# write out to imputer
fwrite(final_test_soil_data_normalized, "normalized_test_soil.csv")

In [499]:
# save imputed soil data
imputed_test_soil_data<- fread("iputed_test_soil_data.csv")
final_test_soil_data <- data.frame("Env" = test_soil_unnormalized$Env,imputed_test_soil_data[,2:length(imputed_test_soil_data)])
colnames(final_test_soil_data) <- colnames(test_soil_unnormalized)

In [500]:
head(final_test_soil_data)

,Env,E Depth,1:1 Soil pH,WDRF Buffer pH,1:1 S Salts mmho/cm,Texture No,Organic Matter LOI %,Nitrate-N ppm N,lbs N/A,Potassium ppm K,⋯,CEC/Sum of Cations me/100g,%H Sat,%K Sat,%Ca Sat,%Mg Sat,%Na Sat,Mehlich P-III ppm P,% Sand,% Silt,% Clay
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,DEH1_2022,0.6666667,0.7951807,1.0000000,0.4038462,1,0.2448980,0.5363036,0.4406780,0.4579125,⋯,0.1067073,0.0000000,0.7692308,0.7582418,0.5757576,1.0,0.5474453,0.8915663,0.2727273,0.2820513
2,GAH1_2022,1.0000000,0.7469880,0.9583333,1.0000000,1,0.1836735,0.4884488,0.6045198,0.7946128,⋯,0.1463415,0.5087719,1.0000000,0.4835165,0.3939394,0.5,1.0000000,1.0000000,0.1818182,0.1794872
3,GAH2_2022,1.0000000,0.7349398,0.9305556,0.4423077,1,0.5306122,0.3118812,0.3841808,0.5016835,⋯,0.2225610,0.7894737,0.3846154,0.4175824,0.3333333,0.5,0.3284672,0.7590361,0.2909091,0.5384615
4,IAH1_2022,0.6666667,0.7469880,0.9166667,0.6923077,1,1.0000000,0.6056106,0.4971751,0.6632997,⋯,0.7530488,0.2631579,0.1538462,0.6483516,0.7272727,0.0,0.3722628,0.1927711,0.8181818,1.0000000
5,IAH2_2022,0.6666667,0.7108434,0.8888889,0.3269231,1,0.7959184,0.2062706,0.1694915,0.6599327,⋯,0.5762195,0.5263158,0.2307692,0.6043956,0.3636364,0.0,0.4306569,0.5301205,0.5272727,0.6923077
6,IAH3_2022,0.6666667,0.7951807,1.0000000,0.5192308,1,0.9795918,0.4339934,0.3559322,0.6531987,⋯,0.5762195,0.0000000,0.2307692,0.8571429,0.5757576,0.0,0.3430657,0.2168675,0.8545455,0.8974359


## EC Data

### training wrangling

In [501]:
# extract columns that are relevant based on correlation
final_ec_data <- select(ec_data, c('Env',ec_relevant$feature))

In [502]:
dim(final_ec_data)

[1] 165 237

### Testing wrangling

In [503]:
final_test_ec_data <- select(test_ec_data, c('Env',ec_relevant$feature))

In [504]:
dim(final_test_ec_data)

[1]  24 237

## Weather Data

### training

In [505]:
colnames(weather_data_filtered)[1] <- 'Env'
head(weather_data_filtered)

Env,T2M_MAX__ratio_beyond_r_sigma__r_1.5,T2M_MAX__ratio_beyond_r_sigma__r_0.5,T2M_MAX__energy_ratio_by_chunks__num_segments_10__segment_focus_8,ALLSKY_SFC_SW_DNI__ratio_beyond_r_sigma__r_2,T2M_MAX__ar_coefficient__coeff_0__k_10,ALLSKY_SFC_SW_DNI__c3__lag_1,"ALLSKY_SFC_PAR_TOT__cwt_coefficients__coeff_13__w_10__widths_(2, 5, 10, 20)","T2M_MAX__agg_autocorrelation__f_agg_""""mean""""__maxlag_40","T2M_MAX__agg_linear_trend__attr_""""intercept""""__chunk_len_50__f_agg_""""min""""",⋯,"PS__fft_coefficient__attr_""""abs""""__coeff_90","QV2M__fft_coefficient__attr_""""abs""""__coeff_30",T2M_MIN__index_mass_quantile__q_0.8,"WS2M__fft_coefficient__attr_""""abs""""__coeff_31","PRECTOTCORR__fft_coefficient__attr_""""abs""""__coeff_39","GWETROOT__agg_linear_trend__attr_""""intercept""""__chunk_len_10__f_agg_""""var""""",ALLSKY_SFC_SW_DNI__index_mass_quantile__q_0.6,"T2MWET__fft_coefficient__attr_""""angle""""__coeff_2","PS__change_quantiles__f_agg_""""var""""__isabs_True__qh_0.8__ql_0.6","PS__fft_coefficient__attr_""""abs""""__coeff_96"
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ARH1_2016,0.10109290,0.6912568,0.08855208,0.002732240,0.9615705,8627.880,44.347083,0.6788986,13.941667,⋯,7.581141,102.57795,0.7295082,36.49787,177.8200,2.269047e-04,0.5901639,-143.9697,0.005307526,1.2398628
ARH1_2017,0.10958904,0.6739726,0.06498877,0.000000000,0.7114158,7344.521,7.314094,0.5658802,11.650000,⋯,9.724787,90.60379,0.7178082,18.07747,202.2883,3.893314e-06,0.5808219,-128.6712,0.009790398,1.8443808
ARH1_2018,0.08493151,0.7342466,0.05039027,0.010958904,0.8562795,6302.137,40.520156,0.6719760,8.171667,⋯,13.162726,35.94472,0.7123288,18.68020,148.2367,1.202191e-04,0.5616438,-134.2286,0.005869679,9.9224612
ARH2_2016,0.10109290,0.6939891,0.08655048,0.005464481,0.9662311,8319.311,51.383579,0.6567808,12.157500,⋯,7.279520,109.54632,0.7185792,30.16200,255.7611,2.301056e-04,0.5901639,-149.4075,0.004273257,0.8713265
ARH2_2017,0.11232877,0.6821918,0.05827578,0.005479452,0.6253540,7497.796,-1.608494,0.5739137,10.731667,⋯,10.393594,79.45131,0.7178082,11.18136,302.6703,4.666572e-05,0.5780822,-136.0726,0.008139840,1.8234584
ARH2_2018,0.09041096,0.7260274,0.04675049,0.010958904,0.8346676,6174.649,36.286837,0.6654573,7.284167,⋯,13.112359,33.75229,0.7095890,25.14146,90.5427,2.385092e-04,0.5589041,-131.5809,0.006395482,10.4020138


### testing

In [518]:
colnames(weather_data_filtered_test)[1] <- 'Env'
weather_data_final_test <- select(weather_data_filtered_test,
       colnames(weather_data_filtered_test)[colnames(weather_data_filtered_test)
                                            %in% colnames(weather_data_filtered)])

## Merging Datasets

### trainging

In [507]:
#subset 3 columns
env_hybrid<- trait_data[,c('Env', 'Hybrid', 'Yield_Mg_ha')]

# rename prs scores
colnames(prs_200) <- c("Hybrid", "PRS200")
colnames(prs_100) <- c("Hybrid", "PRS100")
colnames(prs_50) <- c("Hybrid", "PRS50")
colnames(prs_10) <- c("Hybrid", "PRS10")

# merge prs tables with hybrid names
merged_prs_table <- env_hybrid %>%
left_join(prs_200,by = c("Hybrid")) %>% 
left_join(prs_100,by = c("Hybrid")) %>% 
left_join(prs_50,by = c("Hybrid")) %>% 
left_join(prs_10,by = c("Hybrid"))

#view data
head(merged_prs_table)
dim(merged_prs_table)

Env,Hybrid,Yield_Mg_ha,PRS200,PRS100,PRS50,PRS10
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DEH1_2014,M0088/LH185,5.721725,441.5250,1852.547,5571.501,-8739.188
DEH1_2014,M0143/LH185,11.338246,450.5554,1949.209,5347.498,-9374.159
DEH1_2014,M0003/LH185,6.540810,451.0282,1755.532,5316.643,-9204.930
DEH1_2014,M0035/LH185,10.366857,452.8043,1980.825,5717.198,-8408.220
DEH1_2014,M0052/LH185,10.908814,441.1833,1693.345,4754.723,-10215.428
DEH1_2014,M0008/LH185,11.510118,441.5120,1670.693,5100.381,-9297.425


[1] 136012      7

In [508]:
# merge prs with meta data ec and soil data
merged_table <- merged_prs_table %>% left_join(final_meta_data,by = c("Env")) %>%
left_join(final_ec_data,by = c("Env")) %>% 
left_join(final_soil_data,by = c("Env")) %>%
left_join(weather_data_filtered,by = c("Env"))

In [509]:
dim(merged_table)

[1] 136012    568

In [510]:
head(merged_table)

fwrite(merged_table, "merged_features.csv")

Env,Hybrid,Yield_Mg_ha,PRS200,PRS100,PRS50,PRS10,Treatment,City,Field,⋯,"PS__fft_coefficient__attr_""""abs""""__coeff_90","QV2M__fft_coefficient__attr_""""abs""""__coeff_30",T2M_MIN__index_mass_quantile__q_0.8,"WS2M__fft_coefficient__attr_""""abs""""__coeff_31","PRECTOTCORR__fft_coefficient__attr_""""abs""""__coeff_39","GWETROOT__agg_linear_trend__attr_""""intercept""""__chunk_len_10__f_agg_""""var""""",ALLSKY_SFC_SW_DNI__index_mass_quantile__q_0.6,"T2MWET__fft_coefficient__attr_""""angle""""__coeff_2","PS__change_quantiles__f_agg_""""var""""__isabs_True__qh_0.8__ql_0.6","PS__fft_coefficient__attr_""""abs""""__coeff_96"
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DEH1_2014,M0088/LH185,5.721725,441.5250,1852.547,5571.501,-8739.188,,Georgetown,27AB,⋯,7.013648,55.79507,0.7123288,5.58717,94.03992,0.000195889,0.569863,96.91994,0.009815104,17.99277
DEH1_2014,M0143/LH185,11.338246,450.5554,1949.209,5347.498,-9374.159,,Georgetown,27AB,⋯,7.013648,55.79507,0.7123288,5.58717,94.03992,0.000195889,0.569863,96.91994,0.009815104,17.99277
DEH1_2014,M0003/LH185,6.540810,451.0282,1755.532,5316.643,-9204.930,,Georgetown,27AB,⋯,7.013648,55.79507,0.7123288,5.58717,94.03992,0.000195889,0.569863,96.91994,0.009815104,17.99277
DEH1_2014,M0035/LH185,10.366857,452.8043,1980.825,5717.198,-8408.220,,Georgetown,27AB,⋯,7.013648,55.79507,0.7123288,5.58717,94.03992,0.000195889,0.569863,96.91994,0.009815104,17.99277
DEH1_2014,M0052/LH185,10.908814,441.1833,1693.345,4754.723,-10215.428,,Georgetown,27AB,⋯,7.013648,55.79507,0.7123288,5.58717,94.03992,0.000195889,0.569863,96.91994,0.009815104,17.99277
DEH1_2014,M0008/LH185,11.510118,441.5120,1670.693,5100.381,-9297.425,,Georgetown,27AB,⋯,7.013648,55.79507,0.7123288,5.58717,94.03992,0.000195889,0.569863,96.91994,0.009815104,17.99277


### testing

In [511]:
# merge prs tables with hybrid names
merged_prs_table_test <- submission_template %>%
left_join(prs_200,by = c("Hybrid")) %>% 
left_join(prs_100,by = c("Hybrid")) %>% 
left_join(prs_50,by = c("Hybrid")) %>% 
left_join(prs_10,by = c("Hybrid"))


#view data
head(merged_prs_table_test)
dim(merged_prs_table_test)

Env,Hybrid,Yield_Mg_ha,PRS200,PRS100,PRS50,PRS10
<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>
DEH1_2022,B14A/OH43,NA,428.8070,1700.52880,4563.8017,-9163.324
DEH1_2022,B37/H95,NA,451.2592,2405.06226,5978.8406,-7356.295
DEH1_2022,B73/MO17,NA,464.5094,2040.95086,5081.1533,-10051.763
DEH1_2022,B73/PHN82,NA,294.0542,-93.53413,-488.6736,-20419.721
DEH1_2022,B73/TX779,NA,399.7856,1688.78275,4064.7850,-11389.545
DEH1_2022,B97/LH244,NA,400.4309,1915.12926,4808.4317,-9810.506


[1] 11555     7

In [519]:
# merge prs with meta data ec and soil data
merged_table_test <- merged_prs_table_test %>% left_join(final_test_meta_data,by = c("Env")) %>%
left_join(final_test_ec_data,by = c("Env")) %>% 
left_join(final_test_soil_data,by = c("Env")) %>%
left_join(weather_data_final_test,by = c("Env"))

In [520]:
dim(merged_table_test)
head(merged_table_test)

fwrite(merged_table_test, "merged_features_test.csv")

[1] 11555   568

Env,Hybrid,Yield_Mg_ha,PRS200,PRS100,PRS50,PRS10,Treatment,City,Field,⋯,"PS__fft_coefficient__attr_""""abs""""__coeff_90","QV2M__fft_coefficient__attr_""""abs""""__coeff_30",T2M_MIN__index_mass_quantile__q_0.8,"WS2M__fft_coefficient__attr_""""abs""""__coeff_31","PRECTOTCORR__fft_coefficient__attr_""""abs""""__coeff_39","GWETROOT__agg_linear_trend__attr_""""intercept""""__chunk_len_10__f_agg_""""var""""",ALLSKY_SFC_SW_DNI__index_mass_quantile__q_0.6,"T2MWET__fft_coefficient__attr_""""angle""""__coeff_2","PS__change_quantiles__f_agg_""""var""""__isabs_True__qh_0.8__ql_0.6","PS__fft_coefficient__attr_""""abs""""__coeff_96"
<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DEH1_2022,B14A/OH43,NA,428.8070,1700.52880,4563.8017,-9163.324,Standard,Georgetown,19 AB,⋯,0.7144936,1.355378,0.8057325,3.8777,0.4998807,0.001785251,0.5923567,141.6233,5.068578e-05,0.6235284
DEH1_2022,B37/H95,NA,451.2592,2405.06226,5978.8406,-7356.295,Standard,Georgetown,19 AB,⋯,0.7144936,1.355378,0.8057325,3.8777,0.4998807,0.001785251,0.5923567,141.6233,5.068578e-05,0.6235284
DEH1_2022,B73/MO17,NA,464.5094,2040.95086,5081.1533,-10051.763,Standard,Georgetown,19 AB,⋯,0.7144936,1.355378,0.8057325,3.8777,0.4998807,0.001785251,0.5923567,141.6233,5.068578e-05,0.6235284
DEH1_2022,B73/PHN82,NA,294.0542,-93.53413,-488.6736,-20419.721,Standard,Georgetown,19 AB,⋯,0.7144936,1.355378,0.8057325,3.8777,0.4998807,0.001785251,0.5923567,141.6233,5.068578e-05,0.6235284
DEH1_2022,B73/TX779,NA,399.7856,1688.78275,4064.7850,-11389.545,Standard,Georgetown,19 AB,⋯,0.7144936,1.355378,0.8057325,3.8777,0.4998807,0.001785251,0.5923567,141.6233,5.068578e-05,0.6235284
DEH1_2022,B97/LH244,NA,400.4309,1915.12926,4808.4317,-9810.506,Standard,Georgetown,19 AB,⋯,0.7144936,1.355378,0.8057325,3.8777,0.4998807,0.001785251,0.5923567,141.6233,5.068578e-05,0.6235284


In [521]:
combined_test_train <- rbind(merged_table,merged_table_test)
fwrite(combined_test_train, "merged_features_test_train.csv")